<a href="https://colab.research.google.com/github/pollyjuice74/REU-LDPC-Project/blob/main/ECCT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/pollyjuice74/ECCT
!git clone https://github.com/pollyjuice74/REU-LDPC-Project
!git clone https://github.com/pollyjuice74/gnn-decoder

!pip install sionna
#!pip install torch torch-geometric

In [10]:
import os
import torch.nn as nn
import copy
# Rename folders
if os.path.exists('REU-LDPC-Project'):
  os.rename('REU-LDPC-Project', 'REU_LDPC_Project')
if os.path.exists('gnn-decoder'):
  os.rename('gnn-decoder', 'gnn_decoder')

In [23]:
for i in enc.pcm:
  print(i)
  break

  (0, 10)	1.0
  (0, 26)	1.0
  (0, 41)	1.0
  (0, 51)	1.0
  (0, 98)	1.0
  (0, 142)	1.0
  (0, 151)	1.0
  (0, 165)	1.0


In [20]:
enc._n_ldpc, enc._k_ldpc, enc.pcm.shape[0]

(780, 150, 630)

In [46]:
from sionna.fec.ldpc.encoding import LDPC5GEncoder
from sionna.fec.ldpc.decoding import LDPC5GDecoder

from REU_LDPC_Project.channel import E2EModel

# from ECCT.Model import ECC_Transformer
from ECCT.Codes import FER, BER # x_pred, x_gt
from ECCT.args import pass_args
from ECCT.Model import *

from gnn_decoder.gnn import LDPC5GGNN

n, k = (100, 90)
args = pass_args()

enc = LDPC5GEncoder(k,n)
dec = LDPC5GDecoder(enc)



class ECC_Transformer(nn.Module):
    def __init__(self, args, encoder, decoder, dropout=0):
        super(ECC_Transformer, self).__init__()
        ####
        code = args.code
        c = copy.deepcopy
        attn = MultiHeadedAttention(args.h, args.d_model)
        ff = PositionwiseFeedForward(args.d_model, args.d_model*4, dropout)

        # 5G Compliant encoder/decoder
        self.encoder5G = encoder
        self.decoder5G = decoder

        self.src_embed = torch.nn.Parameter(torch.empty(self.encoder5G._n_ldpc + self.encoder5G.pcm.shape[0], args.d_model)) ### #(code.n + code.pc_matrix.size(0), args.d_model)))
        self.decoder = Encoder(EncoderLayer(
            args.d_model, c(attn), c(ff), dropout), args.N_dec)
        self.oned_final_embed = torch.nn.Sequential(
            *[nn.Linear(args.d_model, 1)])
        self.out_fc = nn.Linear(self.encoder5G._n_ldpc + self.encoder5G.pcm.shape[0], self.encoder5G._n_ldpc) ###

        self.get_mask(code)
        print(f'Mask:\n {self.src_mask}')

        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, magnitude, syndrome):
        emb = torch.cat([magnitude, syndrome], -1).unsqueeze(-1)
        emb = self.src_embed.unsqueeze(0) * emb
        emb = self.decoder(emb, self.src_mask)
        return self.out_fc(self.oned_final_embed(emb).squeeze(-1))

    def loss(self, z_pred, z2, y):
        loss = F.binary_cross_entropy_with_logits(
            z_pred, sign_to_bin(torch.sign(z2)))
        x_pred = sign_to_bin(torch.sign(-z_pred * torch.sign(y)))
        return loss, x_pred

    def get_mask(self, code, no_mask=False):
        if no_mask:
            self.src_mask = None
            return

        def build_mask(code, encoder):
            mask_size = self.encoder5G._n_ldpc +self.encoder5G.pcm.shape[0] # n + m
            mask = torch.eye(mask_size, mask_size)

            for ii in range(self.encoder5G.pcm.shape[0]): # m
                # edge idxs for cn ii in pcm
                idx = self.encoder5G.pcm[ii].indices #torch.where(self.encoder5G.pcm[ii] > 0)[0] #code.pc_matrix[ii]
                print(ii, self.encoder5G.pcm[ii].indices, idx)
                print(self.encoder5G._n_ldpc, self.encoder5G._k_ldpc)
                print()

                for jj in idx:
                    for kk in idx:

                        print(mask.shape)
                        if jj != kk:
                            mask[jj, kk] += 1
                            mask[kk, jj] += 1
                            mask[self.encoder5G._n + ii, jj] += 1
                            mask[jj, self.encoder5G._n + ii] += 1

            src_mask = ~ (mask > 0).unsqueeze(0).unsqueeze(0)
            return src_mask

        src_mask = build_mask(code, self.encoder5G)
        print(src_mask)

        mask_size = self.encoder5G._n_ldpc +self.encoder5G.pcm.shape[0]
        a = mask_size ** 2
        print(
            f'Self-Attention Sparsity Ratio={100 * torch.sum((src_mask).int()) / a:0.2f}%, Self-Attention Complexity Ratio={100 * torch.sum((~src_mask).int())//2 / a:0.2f}%')
        self.register_buffer('src_mask', src_mask)



# Models
gnn = LDPC5GGNN(enc)
ecct = ECC_Transformer(args, enc, dec)

model = None
ch = E2EModel(enc, dec, model)




Streaming output truncated to the last 5000 lines.
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
355 [  0  59  76 505] [  0  59  76 505]
780 150

torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
356 [  1  45  77 506] [  1  45  77 506]
780 150

torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
torch.Size([1410, 1410])
t

In [ ]:
ecct.